*Installing pulp which is a linear programming library in Python*

In [19]:
pip install pulp

# **Problem 1 - Sitting Sales Representatives**

In [20]:
# import pulp
import pulp

# Defining the question
question = pulp.LpProblem("Sales_Representative_Assignment", pulp.LpMinimize)

# Defining decision variables

# A list of integer variables representing the number of sales representatives in each of the 4 districts.
num_reps = [pulp.LpVariable(f'num_reps_{i}', lowBound=0, cat='Integer') for i in range(4)]

# A 2D list of continuous variables representing the number of calls made from each district to each district.
num_calls = [[pulp.LpVariable(f'calls_{i}_{j}', lowBound=0, cat='Continuous') for j in range(4)] for i in range(4)]

# A list of binary variables indicating whether fixed costs apply to each district (1 if at least one representative is assigned, 0 otherwise).
fixed_cost_indicators = [pulp.LpVariable(f'fixed_cost_{i}', cat='Binary') for i in range(4)]

# Define the cost coefficients
# The fixed cost for assigning at least one representative to a district.
fixed_cost = 88000

# The variable cost per representative.
variable_cost = 80000

# Defining the time matrix, a matrix representing the time (in hours) required for calls between districts.
time_matrix = [
    [1, 4, 5, 7],
    [4, 1, 3, 5],
    [5, 3, 1, 2],
    [7, 5, 2, 1]]

# Defining a list of the required number of calls each district needs to receive.
required_calls = [50, 80, 100, 60]

# Objective function: Minimize the total cost, the total cost, which is the sum of fixed and variable costs for each district.
total_cost = sum(fixed_cost_indicators[i] * fixed_cost + variable_cost * num_reps[i] for i in range(4))

# Adding the objective function to the problem.
question += total_cost

# Constraints for meeting the required number of calls, this ensures that each district receives at least the required number of calls.
for j in range(4):
    question += sum(num_calls[i][j] for i in range(4)) >= required_calls[j]

# Constraints for the working hours of each sales representative
# Ensuring that the total working hours for representatives in each district do not exceed 160 hours per representative and that if a fixed cost is incurred, there is at least one representative assigned to the district.
for i in range(4):
    question += sum(time_matrix[i][j] * num_calls[i][j] for j in range(4)) <= 160 * num_reps[i]
    question += num_reps[i] >= fixed_cost_indicators[i]

# Ensuring the binary variable correctly reflects if there are any representatives
for i in range(4):
    question += num_reps[i] <= 100 * fixed_cost_indicators[i]  # Assuming a high enough number of reps as upper bound

# Solving the problem
question.solve()

# Printing the results in a readable format
print("Number of sales representatives assigned to each district:")
for i in range(4):
    print(f"District {i+1}: {num_reps[i].varValue} representatives")

print("\nNumber of calls made by representatives from each district to each district:")
for i in range(4):
    for j in range(4):
        print(f"Calls from District {i+1} to District {j+1}: {num_calls[i][j].varValue}")
print(f"\nTotal Cost: ${pulp.value(question.objective):,.2f}")


Number of sales representatives assigned to each district:
District 1: 0.0 representatives
District 2: 0.0 representatives
District 3: 5.0 representatives
District 4: 0.0 representatives

Number of calls made by representatives from each district to each district:
Calls from District 1 to District 1: 0.0
Calls from District 1 to District 2: 0.0
Calls from District 1 to District 3: 0.0
Calls from District 1 to District 4: 0.0
Calls from District 2 to District 1: 0.0
Calls from District 2 to District 2: 0.0
Calls from District 2 to District 3: 0.0
Calls from District 2 to District 4: 0.0
Calls from District 3 to District 1: 50.0
Calls from District 3 to District 2: 80.0
Calls from District 3 to District 3: 100.0
Calls from District 3 to District 4: 60.0
Calls from District 4 to District 1: 0.0
Calls from District 4 to District 2: 0.0
Calls from District 4 to District 3: 0.0
Calls from District 4 to District 4: 0.0

Total Cost: $488,000.00


# **Problem 2 - The Wild Turkey Company**

In [21]:
# Defining the question
question = pulp.LpProblem("Maximize Wild Turkey's Profit", pulp.LpMaximize)

# Defining the variables
# cutlet_1_produced: pounds of cutlet 1 produced
cutlet_1_produced = pulp.LpVariable('x1', lowBound=0, cat='Continuous') # lowbound =0 as production cannot be negative, cat=Continuous as these variables can take any value

# cutlet_2_produced: pounds of cutlet 2 produced
cutlet_2_produced = pulp.LpVariable('x2', lowBound=0, cat='Continuous') # lowbound =0 as production cannot be negative, cat=Continuous as these variables can take any value

# turkey_1_purchased: number of type 1 turkeys purchased
turkey_1_purchased = pulp.LpVariable('t1', lowBound=100, cat='Integer')  # Minimum purchase constraint included here, category=Integer as the number of turkeys purchased cannot be a fraction

# turkey_2_purchased: number of type 2 turkeys purchased
turkey_2_purchased = pulp.LpVariable('t2', lowBound=0, cat='Integer') # No minimum purchase constraint included here, category=Integer as the number of turkeys purchased cannot be a fraction

# Objective Function: Maximize profitability
profit = 4 * cutlet_1_produced + 3 * cutlet_2_produced - 10 * turkey_1_purchased - 8 * turkey_2_purchased
question += profit

# Total white and dark meat available from purchased turkeys
white_meat = 5 * turkey_1_purchased + 3 * turkey_2_purchased
dark_meat = 2 * turkey_1_purchased + 3 * turkey_2_purchased

# White meat constraints (no dark meat constraints required)
question += cutlet_1_produced * 0.7 + cutlet_2_produced * 0.6 <= white_meat  # Total white meat used in both cutlets

# Total meat constraints
question += cutlet_1_produced + cutlet_2_produced <= white_meat + dark_meat  # Ensure we have enough total meat for both cutlets

# Minimum purchase constraint
question += turkey_1_purchased >=100

# Capacity constraints for cutlet 1 and cutlet 2
question += cutlet_1_produced <= 5000
question += cutlet_2_produced <= 3000

# Solve the problem
question.solve()

# Output results
print(f"Number of pounds of cutlet 1 produced: {cutlet_1_produced.varValue}")
print(f"Number of pounds of cutlet 2 produced: {cutlet_2_produced.varValue}")
print(f"Number of type 1 turkeys purchased: {turkey_1_purchased.varValue}")
print(f"Number of type 2 turkeys purchased: {turkey_2_purchased.varValue}")
print(f"Total profit: ${pulp.value(question.objective)}")


Number of pounds of cutlet 1 produced: 5000.0
Number of pounds of cutlet 2 produced: 3000.0
Number of type 1 turkeys purchased: 867.0
Number of type 2 turkeys purchased: 322.0
Total profit: $17754.0


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


# **Problem 3 - Kingston Manufacturing**

In [22]:
import pulp as lp

# Defining the problem
prob = lp.LpProblem("Kingston_Manufacturing", lp.LpMinimize)

# Decision variables as specified in the problem statement
weeks = range(1, 10)
P_production = lp.LpVariable.dicts("P_production", weeks, 0)
H_production = lp.LpVariable.dicts("H_production", weeks, 0)
P_inventory = lp.LpVariable.dicts("P_inventory", weeks, 0)
H_inventory = lp.LpVariable.dicts("H_inventory", weeks, 0)
changeover = lp.LpVariable.dicts("changeover", weeks, 0, 1, lp.LpBinary)
produce_P = lp.LpVariable.dicts("produce_P", weeks, 0, 1, lp.LpBinary)

# Constants as specified in the problem statement
demand_P = {1: 55, 2: 55, 3: 44, 4: 0, 5: 45, 6: 45, 7: 36, 8: 35, 9: 35}
demand_H = {1: 38, 2: 38, 3: 30, 4: 0, 5: 48, 6: 48, 7: 58, 8: 57, 9: 58}
P_initial_inventory = 125
H_initial_inventory = 143
P_production_capacity = 100
H_production_capacity = 80
P_production_cost = 225
H_production_cost = 310
changeover_cost = 500
inventory_cost_rate = 0.195 / 52  # Weekly rate

# Objective function components
production_cost = lp.lpSum(P_production[i] * P_production_cost + H_production[i] * H_production_cost for i in weeks)
inventory_cost = lp.lpSum(P_inventory[i] * P_production_cost * inventory_cost_rate + H_inventory[i] * H_production_cost * inventory_cost_rate for i in weeks)
changeover_cost = lp.lpSum(changeover[i] * 500 for i in weeks)

prob += production_cost + inventory_cost + changeover_cost

# Constraints as specified in the problem statement
for i in weeks:
    if i == 1:
        prob += P_inventory[i] == P_initial_inventory + P_production[i] - demand_P[i]
        prob += H_inventory[i] == H_initial_inventory + H_production[i] - demand_H[i]
    else:
        prob += P_inventory[i] == P_inventory[i-1] + P_production[i] - demand_P[i]
        prob += H_inventory[i] == H_inventory[i-1] + H_production[i] - demand_H[i]

    # Production capacity constraints
    prob += P_production[i] <= produce_P[i] * P_production_capacity
    prob += H_production[i] <= (1 - produce_P[i]) * H_production_capacity

    # Changeover constraints
    if i > 1:
        prob += changeover[i] >= produce_P[i] - produce_P[i-1]
        prob += changeover[i] >= produce_P[i-1] - produce_P[i]

    # Safety stock constraints
    if i < 9:
        prob += P_inventory[i] >= 0.8 * demand_P[i+1]
        prob += H_inventory[i] >= 0.8 * demand_H[i+1]

# Solving the problem
prob.solve()

# Displaying results
print("Week | P-Production | H-Production | P-Inventory | H-Inventory | Changeover")
print("---------------------------------------------------------------")
for i in weeks:
    print(f"{i:>4} | {P_production[i].varValue:>12} | {H_production[i].varValue:>12} | {P_inventory[i].varValue:>11} | {H_inventory[i].varValue:>11} | {int(changeover[i].varValue):>10}")

# Calculating and printing the cost breakdown
total_production_cost = lp.value(production_cost)
total_inventory_cost = lp.value(inventory_cost)
total_changeover_cost = lp.value(changeover_cost)
total_cost = lp.value(prob.objective)

print("\nCost Breakdown:")
print(f"Total Production Cost: ${total_production_cost:.2f}")
print(f"Total Inventory Cost: ${total_inventory_cost:.2f}")
print(f"Total Changeover Cost: ${total_changeover_cost:.2f}")
print(f"Total Cost: ${total_cost:.2f}")


Week | P-Production | H-Production | P-Inventory | H-Inventory | Changeover
---------------------------------------------------------------
   1 |         25.0 |          0.0 |        95.0 |       105.0 |          0
   2 |        100.0 |          0.0 |       140.0 |        67.0 |          0
   3 |        100.0 |          0.0 |       196.0 |        37.0 |          0
   4 |          0.0 |          1.4 |       196.0 |        38.4 |          1
   5 |          0.0 |         48.0 |       151.0 |        38.4 |          0
   6 |          0.0 |         56.0 |       106.0 |        46.4 |          0
   7 |          0.0 |         57.2 |        70.0 |        45.6 |          0
   8 |          0.0 |         57.8 |        35.0 |        46.4 |          0
   9 |          0.0 |         11.6 |         0.0 |         0.0 |          0

Cost Breakdown:
Total Production Cost: $122545.00
Total Inventory Cost: $1327.60
Total Changeover Cost: $500.00
Total Cost: $124372.60


# **Problem 4 - Assigning Technicians to Shifts**

In [23]:
import pulp
import pandas as pd

# Defining the variables as specified in the problem statement
technicians = [1, 2, 3, 4, 5, 6, 7, 8]
shifts = [1, 2, 3]
categories = [1, 2, 3, 4]

# Costs of assigning each technician to each shift
costs = {
    (1, 1): 280, (1, 2): 290, (1, 3): 300,
    (2, 1): 300, (2, 2): 330, (2, 3): 350,
    (3, 1): 270, (3, 2): 280, (3, 3): 290,
    (4, 1): 180, (4, 2): 190, (4, 3): 200,
    (5, 1): 175, (5, 2): 160, (5, 3): 185,
    (6, 1): 225, (6, 2): 225, (6, 3): 225,
    (7, 1): 295, (7, 2): 295, (7, 3): 315,
    (8, 1): 305, (8, 2): 350, (8, 3): 320
}

# Certification categories for each technician
certifications = {
    1: [1, 2, 4],
    2: [2, 3, 4],
    3: [3, 4],
    4: [1],
    5: [2],
    6: [2, 4],
    7: [1, 2, 3],
    8: [2, 4]
}

# Creating the Linear Programming problem
lp = pulp.LpProblem("Technician_Shift_Assignment", pulp.LpMinimize)

# Creating the decision variables
x = pulp.LpVariable.dicts("x", [(i, j) for i in technicians for j in shifts], cat='Binary')

# Objective function: Minimizing the total cost
lp += pulp.lpSum([costs[(i, j)] * x[(i, j)] for i in technicians for j in shifts])

# Constraints: Each technician is assigned to at most one shift
for i in technicians:
    lp += pulp.lpSum([x[(i, j)] for j in shifts]) <= 1

# Constraints: Each shift has at least one certified technician in each category
for j in shifts:
    for c in categories:
        lp += pulp.lpSum([x[(i, j)] for i in technicians if c in certifications[i]]) >= 1

# Solving the Linear Programming problem
lp.solve()

# Outputting the results
print("Status:", pulp.LpStatus[lp.status])
assignments = []
total_cost = 0
for j in shifts:
    print(f"Shift {j}:")
    shift_cost = 0
    for i in technicians:
        if x[(i, j)].varValue == 1:
            print(f"  Technician {i} (Cost: ${costs[(i, j)]})")
            assignments.append((i, j, costs[(i, j)]))
            shift_cost += costs[(i, j)]
    print(f"  Total cost for Shift {j}: ${shift_cost}")
    total_cost += shift_cost

# Calculating total cost
print(f"Total cost: ${total_cost}")

# Displaying the assignments
assignments_df = pd.DataFrame(assignments, columns=['Technician', 'Shift', 'Cost'])
print(assignments_df)


Status: Optimal
Shift 1:
  Technician 2 (Cost: $300)
  Technician 4 (Cost: $180)
  Total cost for Shift 1: $480
Shift 2:
  Technician 6 (Cost: $225)
  Technician 7 (Cost: $295)
  Total cost for Shift 2: $520
Shift 3:
  Technician 1 (Cost: $300)
  Technician 3 (Cost: $290)
  Total cost for Shift 3: $590
Total cost: $1590
   Technician  Shift  Cost
0           2      1   300
1           4      1   180
2           6      2   225
3           7      2   295
4           1      3   300
5           3      3   290
